In [1]:
#!pip install nltk
#!pip install tensorflow
#!pip install keras
import sklearn as sk
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
import string
import nltk
import tensorflow
from sklearn.model_selection import train_test_split
from collections import Counter
from pandas import DataFrame
from matplotlib import pyplot
import re
from nltk.stem import WordNetLemmatizer 
from keras.preprocessing.text import Tokenizer

#import sys/
#!conda install --yes --prefix {sys.prefix} anaconda bcolz 
import bcolz

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nimmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def process_news(news):
    _news = news.replace('b\"', "")
    _news = _news.replace('b\'', "")
    _news = _news.lower()
    _news = re.sub("[^a-zA-Z]", " ",_news)
    _news = re.sub('[\s]+', ' ', _news)
    
    tokens = _news.split(" ")
    if "" in tokens:
        tokens.remove("")
    
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    #remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    _news = ' '.join(tokens)    
     
    return _news


def read_data():

    data = pd.read_csv("../Datasets/djia/Combined_News_DJIA.csv")
    
    dfs = []
    data["News"] = ""
    for i in range(1,25):
        col = "Top"+str(i)
        data["News"] = data["News"] +" "+ data[col]
    data = data.dropna()
    data['PreProcessedNews'] = data['News'].map(process_news)
    
    data = data[['Date', 'News', 'PreProcessedNews', 'Label']]
    
    stock_prices = "../Datasets/djia/upload_DJIA_table.csv"
    stock_data = pd.read_csv(stock_prices)
    
    print(data.head(2))
    print(stock_data.head(2))
    
    
    #merged_dataframe = data.merge(stock_data, how='inner', on='Date')
    merged_dataframe = pd.merge(data, stock_data, how='inner', on = 'Date')

    
    
    
    Xy_train = merged_dataframe[:int(len(data)*0.6)]
    Xy_valid = merged_dataframe[int(len(data)*0.6):int(len(data)*0.8)]
    Xy_test = merged_dataframe[int(len(data)*0.8):]
    
    return merged_dataframe, Xy_train, Xy_valid, Xy_test

In [3]:
news, Xy_train, Xy_valid, Xy_test = read_data()

         Date                                               News  \
0  2008-08-08   b"Georgia 'downs two Russian warplanes' as co...   
1  2008-08-11   b'Why wont America and Nato help us? If they ...   

                                    PreProcessedNews  Label  
0  georgia two russian warplane country move brin...      0  
1  wont america nato help wont help help iraq bus...      1  
         Date          Open          High           Low         Close  \
0  2016-07-01  17924.240234  18002.380859  17916.910156  17949.369141   
1  2016-06-30  17712.759766  17930.609375  17711.800781  17929.990234   

      Volume     Adj Close  
0   82160000  17949.369141  
1  133030000  17929.990234  


## Non data-dependent method
Predictor which always predict up

In [4]:
tp = len(Xy_test[Xy_test['Label']==1])
tn = 0
fn = 0
fp = len(Xy_test[Xy_test['Label']==0])

In [5]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f_measure = (2 * precision * recall) / (precision + recall)
accuracy = (tp+tn)/(tp+tn+fp+fn)

In [6]:
print(precision)
print(recall)
print(f_measure)
print(accuracy)

0.507537688442211
1.0
0.6733333333333333
0.507537688442211


## Data-dependent method
The predictor which always predict the previous day trend.

In [15]:
len(Xy_test)

398

In [20]:
len(np.concatenate((Xy_valid['Label'].values[-1:], Xy_test['Label'].values[:-1])))

398

In [24]:
pred = np.concatenate((Xy_valid['Label'].values[-1:], Xy_test['Label'].values[:-1]))

In [25]:
Xy_test['Prediction'] = pred

<ipython-input-25-d8bcc895ea64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xy_test['Prediction'] = pred


In [30]:
Xy_test[['Label', 'Prediction']]

,Label,Prediction
1588,1,1
1589,0,1
1590,1,0
1591,0,1
1592,0,0
...,...,...
1981,0,0
1982,1,0
1983,1,1
1984,1,1


In [35]:
tp = len(Xy_test[(Xy_test['Label']==1) & (Xy_test['Prediction']==1)])
fp = len(Xy_test[(Xy_test['Label']==0) & (Xy_test['Prediction']==1)])
tn = len(Xy_test[(Xy_test['Label']==0) & (Xy_test['Prediction']==0)])
fn = len(Xy_test[(Xy_test['Label']==1) & (Xy_test['Prediction']==0)])

In [36]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f_measure = (2 * precision * recall) / (precision + recall)
accuracy = (tp+tn)/(tp+tn+fp+fn)

In [37]:
print(precision)
print(recall)
print(f_measure)
print(accuracy)

0.4752475247524752
0.4752475247524752
0.4752475247524752
0.46733668341708545
